In [2]:
import uproot as ur
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak

## Get data, transform ROOT tree into array

In [ ]:
energy = 1 # Energy in GeV
files = [] # List to contain all the data file names

# Loop through some large number to obtain all the available files so we don't have to look them up manually. 
for file_number in range(1, 201):
    try:
        file_name = 'root://sci-xrootd.jlab.org//osgpool/eic/ATHENA/RECO/SINGLE/pi+\
/{energy}GeV/3to50deg/pi+_{energy}GeV_3to50deg.{file_number:04d}.root:events'.format(energy = energy, file_number = file_number)
        ur.open(file_name) # Open each file. If the file doesn't exist, it will be skipped and not be added to files list.
        files.append(file_name)
    except:
        continue
        
print("Number of files: {NumFiles}.".format(NumFiles = len(files)))

# Combining all the data files that we found
# Cannot combine each desired branch into one big array. Have to either combine all branches (including ones we don't want like Barrel branches) or make separate arrays for each branch we want.
EcalN_events = ur.concatenate(files, ["EcalEndcapNClusters"])
EcalP_events = ur.concatenate(files, ["EcalEndcapPClusters"])
HcalHadron_events = ur.concatenate(files, ["HcalHadronEndcapClusters"])

### Next steps:
### 1) Filter the angles. Choose a certain angle, then fill arrays of edep for that angle
### 2) Fill a weighting histogram and fit it with a Gaussian to obtain the optimal weight
### 3) Fill a new histogram with the optimally weighted energy dependence and fit it to obtain the energy resolution
### 4) Do this for each energy and then plot (like EIC_EndCap.C)
